In [30]:
import pandas as pd
import pyterrier as pt
import numpy as np
import os
from repro_eval.Evaluator import RpdEvaluator
from repro_eval.util import arp, arp_scores
import pytrec_eval
import yaml
if not pt.started():
    pt.init()

# Time Fuse
Fuse the new run with one old run by boosting old (known) documents up and new documents down.

In [31]:
def time_fuse(run_recent, run_old, _lambda=0.5):
    qid_ranking_groups = run_old.groupby('qid')
    qid_ranking_dict = {qid: list(ranking['docno']) for qid, ranking in qid_ranking_groups}
    
    def weigh(row):
        if not qid_ranking_dict.get(row['qid']):
            print("Could not find", row['qid'])
            
        if row['docno'] in qid_ranking_dict.get(row['qid'], []):
            return row['score'] * _lambda ** 2
        else:
            return row['score'] * (1-_lambda) ** 2              
    reranking = run_recent.copy()
    
    # min max normalization per topic
    reranking['score'] = reranking.groupby('qid')['score'].transform(lambda x : x / x.max())
    
    # weight if in old ranking
    reranking['score'] = reranking.progress_apply(weigh, axis=1)
    reranking = reranking.sort_values(['qid','score'], ascending=False).groupby('qid').head(1000)
    reranking['rank'] = reranking.groupby('qid')['score'].rank(ascending=False).astype(int)
    return reranking

# Lambda sweep

In [32]:
base_path = "data"
runs_path = "results/trec"
reranked_path = "results/fuse_time"

run_new_path = "CIR_BM25_D-t3_T-t3"
run_old_path = "CIR_BM25_D-t2_T-t3_extended"

with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [33]:
run_new = pt.io.read_results(os.path.join(base_path, runs_path, run_new_path))
run_old = pt.io.read_results(os.path.join(base_path, runs_path, run_old_path))

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

results = {}
for l in np.logspace(-3, -7, num=10):  # sweep from 1e-3 to 1e-12
    _lambda = 0.5 + l
    run_reranked = time_fuse(run_new, run_old, _lambda=_lambda)
    results[_lambda] = {}
    
    # write results
    run_name = f'CIR_BM25_D-t3_T-t3_rr-t2-{l}'
    run_reranked_path = os.path.join(base_path, reranked_path, run_name)
    pt.io.write_results(run_reranked, run_reranked_path, format='trec', run_name=run_name)

    # evaluate
    rpd_eval = RpdEvaluator(run_b_orig_path=os.path.join(base_path, runs_path, run_new_path), run_b_rep_path=run_reranked_path)
    
    correlations = rpd_eval.ktau_union().get('baseline')
    correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]
    avg_tau = sum(correlation_scores) / len(correlation_scores)
    results[_lambda]["tau"] = avg_tau
    
    with open(run_reranked_path) as run_reranked:
        run = pytrec_eval.parse_run(run_reranked)
        scores = evaluator.evaluate(run)
        results[_lambda]["arp"] = arp_scores(scores)

 43%|████▎     | 251596/585414 [00:02<00:02, 117883.31it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 252727/585414 [00:02<00:02, 120243.76it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 252282/585414 [00:02<00:02, 119169.18it/s]

...

 43%|████▎     | 253221/585414 [00:02<00:02, 121136.06it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 252335/585414 [00:02<00:02, 119197.97it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


100%|██████████| 585414/585414 [00:05<00:00, 111763.08it/s]


In [17]:
for l in results.keys():
    print("|", " | ".join([
        str(l), 
        str(results[l]["tau"]), 
        str(results[l]["arp"]["P_10"]), 
        str(results[l]["arp"]["bpref"]), 
        str(results[l]["arp"]["ndcg"])
        ]), "|")

| 0.501 | 0.07894661726059203 | 0.16521739130434782 | 0.4369563935417115 | 0.36664537840465017 |
| 0.5003593813663805 | 0.18015837174710114 | 0.1632107023411371 | 0.43696681591301645 | 0.3651031532532713 |
| 0.5001291549665015 | 0.3673963392321741 | 0.16270903010033447 | 0.4370746648615697 | 0.36422704548714674 |
| 0.5000464158883361 | 0.5920213803430702 | 0.16270903010033447 | 0.43733111170037436 | 0.36391700223004286 |
| 0.500016681005372 | 0.7697204337114719 | 0.1623745819397993 | 0.43735522934463766 | 0.3638098161655337 |
| 0.5000059948425032 | 0.8711087893364641 | 0.16237458193979934 | 0.4373071666412955 | 0.36375400554466014 |
| 0.50000215443469 | 0.9201042128690045 | 0.16237458193979934 | 0.43728103787875366 | 0.36375733241888364 |
| 0.5000007742636827 | 0.9414481394214028 | 0.16237458193979934 | 0.43728103787875366 | 0.36375625919355037 |
| 0.5000002782559402 | 0.9499184826743542 | 0.16237458193979934 | 0.43728103787875366 | 0.363756441514514 |
| 0.5000001 | 0.9530700020867431 

In [18]:
# raw ranking
with open(os.path.join(base_path, runs_path, run_new_path)) as run_new:
    run = pytrec_eval.parse_run(run_new)
    scores = evaluator.evaluate(run)
    print(arp_scores(scores)["P_10"])
    print(arp_scores(scores)["bpref"])
    print(arp_scores(scores)["ndcg"])

0.1623745819397993
0.43728103787875366
0.36375525364044387


# Filter Fuse

In [19]:
import numpy as np
from repro_eval.Evaluator import RpdEvaluator
from ranx import Run, fuse

In [20]:
def filter_and_fuse(run_recent, old_runs: list):
    qid_ranking_groups = run_recent.groupby('qid')
    qid_ranking_dict_recent = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
    
    runs = [Run.from_dict(qid_ranking_dict_recent)]
    
    for run_old in old_runs:
        qid_ranking_groups = run_old.groupby('qid')
        qid_ranking_dict_old = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
        for qid, ranking in qid_ranking_dict_old.items():
            docs_recent = qid_ranking_dict_recent.get(qid).keys()
            qid_ranking_dict_old[qid] = {docid: score for docid, score in ranking.items() if docid in docs_recent}
        runs.append(Run.from_dict(qid_ranking_dict_old))
    
    combined_run = fuse(runs = runs, method = "rrf")

    return combined_run

In [21]:
base_path = "data"
runs_path = "results/trec"
reranked_path = "results/filter_fuse"

run_new_path = "CIR_BM25_D-t3_T-t3"
run_old_path = "CIR_BM25_D-t2_T-t3_extended"

with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [22]:
run_new = pt.io.read_results(os.path.join(base_path, runs_path, run_new_path))

old_runs = []
for name in os.listdir(os.path.join(base_path, runs_path)):
    if "T-t3" in name and name.endswith("extended"):
        run = pt.io.read_results(os.path.join(base_path, runs_path, name))
        old_runs.append(run)

In [23]:
# find core qids
topic_sets = []
for i in old_runs:
    topic_sets.append(set(i["qid"]))
topic_sets.append(set(run_new["qid"]))

core = set.intersection(*topic_sets)
print(len(core))

597


In [24]:
old_runs_cleaned = []
for run in old_runs:
    old_runs_cleaned.append(run[run["qid"].isin(core)])
    

In [25]:
old_runs = old_runs_cleaned

In [26]:
run_new_cleaned = run_new[run_new["qid"].isin(core)]

In [27]:
run_reranked = filter_and_fuse(run_new_cleaned, old_runs)

In [28]:
run_name = f'CIR_BM25_D-t3_T-t3_rr-ff'
run_reranked_path = os.path.join(base_path, reranked_path, run_name)
run_reranked.save(run_reranked_path,  kind='trec')

In [29]:
rpd_eval = RpdEvaluator(run_b_orig_path=os.path.join(base_path, runs_path, run_new_path), run_b_rep_path=run_reranked_path)

correlations = rpd_eval.ktau_union().get('baseline')
correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]

print("Avg. Kendall's tau: ", sum(correlation_scores) / len(correlation_scores))

with open(run_reranked_path) as run_reranked:
    run = pytrec_eval.parse_run(run_reranked)
    scores = evaluator.evaluate(run)
    print(arp_scores(scores)["bpref"])
    print(arp_scores(scores)["P_10"])
    print(arp_scores(scores)["ndcg"])

Avg. Kendall's tau:  0.004242197558995947
0.4218068440668467
0.10619765494137354
0.29136889967380725
